# Analysing Tartu Smart Bike usage data 2019-2021
Aleksander Tamm, Martin Vendelin, Otto Jõeleht

In [33]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from datetime import datetime

In [27]:
data2019 = pd.read_csv("2019.csv", sep=',', header=0)
data2020 = pd.read_csv("2020.csv", sep=',', header=0)
data2021 = pd.read_csv("2021.csv", sep=',', header=0)
dokid= ...
#kogudata= data2019.append(data2020.append(data2021))
#parse_dates = ['unlockedattime', 'unlockedat',"lockedat","lockedattime"]




In [51]:
#andmete puhastus valedest dokkidest ja lühikestest sõitudest

data2019 = data2019.drop(data2019[(data2019.startstationname == 'Warehouse') | (data2019.endstationname == 'Warehouse')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == 'Lauluväljak') | (data2019.endstationname == 'Lauluväljak')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == 'SPARK Demo') | (data2019.endstationname == 'SPARK Demo')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == 'Töökoda') | (data2019.endstationname == 'Töökoda')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "Tour d'ÖÖ") | (data2019.endstationname == "Tour d'ÖÖ")].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "-- Määramata --") | (data2019.endstationname == "-- Määramata --")].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "Metallica parkla") | (data2019.endstationname == "Metallica parkla")].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "Undetermined") | (data2019.endstationname == "Undetermined")].index)
data2019 = data2019.drop(data2019[data2019['length'] < 0.05].index)
data2019 = data2019.drop(data2019[data2019.length == 0].index)

data2020 = data2020.drop(data2020[(data2020.startstationname == '-- Määramata --') | (data2020.endstationname == '-- Määramata --')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Rahu 8') | (data2020.endstationname == 'Rahu 8')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'SPARK Demo') | (data2020.endstationname == 'SPARK Demo')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Töökoda') | (data2020.endstationname == 'Töökoda')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Undetermined') | (data2020.endstationname == 'Undetermined')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Ihaste Päev') | (data2020.endstationname == 'Ihaste Päev')].index)
data2020 = data2020.drop(data2020[data2020['length'] < 0.05].index)
data2020 = data2020.drop(data2020[data2020.length == 0].index)

data2021 = data2021.drop(data2021[(data2021.startstationname == 'SPARK Demo') | (data2021.endstationname == 'SPARK Demo')].index)
data2021 = data2021.drop(data2021[(data2021.startstationname == '-- Määramata --') | (data2021.endstationname == '-- Määramata --')].index)
data2021 = data2021.drop(data2021[(data2021.startstationname == 'Töökoda') | (data2021.endstationname == 'Töökoda')].index)
data2021 = data2021.drop(data2021[(data2021.startstationname == 'Undetermined') | (data2021.endstationname == 'Undetermined')].index)
data2021 = data2021.drop(data2021[data2021['length'] < 0.05].index)
data2021 = data2021.drop(data2021[data2021.length == 0].index)


#andmed 1te tabelisse
kogudata= data2019.append(data2020.append(data2021))
kogudata['unlocktime'] = kogudata.apply(lambda row: row.unlockedat.strip() +"/"+ row.unlockedattime.strip(), axis=1)
kogudata['locktime'] = kogudata.apply(lambda row: row.lockedat.strip() +"/"+ row.lockedattime.strip(), axis=1)
kogudata=kogudata.drop(columns=["startstationserialnumber","endstationserialnumber","unlockedat","unlockedattime","lockedat","lockedattime"])
kogudata['unlocktime']  = pd.to_datetime(kogudata['unlocktime'] ,errors='coerce', format="%Y-%m-%d/%H:%M:%S")
kogudata['locktime']  = pd.to_datetime(kogudata['locktime'] ,errors='coerce', format="%Y-%m-%d/%H:%M:%S")

In [52]:
kogudata


,cyclenumber,startstationname,endstationname,length,yearOfBirth,first3IdNumber,unlocktime,locktime
3,2421.0,Pirogovi plats,Pirogovi plats,0.96,NaN,NaN,2019-06-03/12:47:10,2019-06-03/12:53:28
5,2290.0,Pirogovi plats,Lossi,1.53,1995.0,395,2019-06-03/12:55:24,2019-06-03/13:06:37
8,2416.0,Pirogovi plats,Pirogovi plats,18.62,NaN,NaN,2019-06-03/12:59:21,2019-06-03/14:55:37
9,2421.0,Pirogovi plats,Pirogovi plats,17.10,NaN,NaN,2019-06-03/13:14:54,2019-06-03/14:56:29
10,2290.0,Lossi,Tulbi,2.04,1981.0,381,2019-06-03/13:18:52,2019-06-03/13:26:50
...,...,...,...,...,...,...,...,...
84999,2514.0,Kalda tee,Kalda tee,6.82,2002.0,602,2021-04-30/23:52:51,2021-05-01/00:28:09
85000,2384.0,Ringtee,ERM,6.69,1973.0,473,2021-04-30/23:53:50,2021-05-01/00:15:46
85001,2227.0,Kalda tee,Põhja puiestee,1.22,1940.0,340,2021-04-30/23:53:53,2021-04-30/23:58:42
85002,2537.0,Põhja puiestee,Sõpruse sild,2.35,NaN,NaN,2021-04-30/23:54:30,2021-05-01/00:02:03


In [43]:
#proov puhastada aegasid
x= pd.read_csv("2021.csv", sep=',', header=0)#, parse_dates=parse_dates)
x['unlocktime'] = x.apply(lambda row: row.unlockedat.strip() +"/"+ row.unlockedattime.strip(), axis=1)
x['locktime'] = x.apply(lambda row: row.lockedat.strip() +"/"+ row.lockedattime.strip(), axis=1)
x=x.drop(columns=["startstationserialnumber","endstationserialnumber","unlockedat","unlockedattime","lockedat","lockedattime"])

def idSünd():
    if(x['yearOfBirth'] not )
    if (first3IdNumber[0] == 3 | first3IdNumber[0] == 4):
        x['yearOfBirth'] = int('19'+first3IdNumber[1:])
    elif(first3IdNumber[0] == 5 | first3IdNumber[0] == 6):
        x['yearOfBirth'] = int('20'+first3IdNumber[1:])
    elif(first3IdNumber[0] == 1 | first3IdNumber[0] == 2):
        x['yearOfBirth'] = int('18'+first3IdNumber[1:])

,cyclenumber,startstationname,endstationname,length,yearOfBirth,first3IdNumber,unlocktime,locktime
0,2581,Aparaaditehas,Rahu,1.73,NaN,601,2021-01-01/06:00:21,2021-01-01/06:10:52
1,2544,Uueturu,Vabadussild,1.10,1997.0,397,2021-01-01/06:21:41,2021-01-01/06:27:36
2,2715,Põhja puiestee,Mõisavahe,2.50,2003.0,603,2021-01-01/07:04:00,2021-01-01/07:24:56
3,2716,Põhja puiestee,Mõisavahe,2.40,2003.0,603,2021-01-01/07:04:31,2021-01-01/07:24:52
4,2548,Turusild,Sõpruse sild,1.60,NaN,499,2021-01-01/07:08:03,2021-01-01/07:17:57
...,...,...,...,...,...,...,...,...
84999,2514,Kalda tee,Kalda tee,6.82,2002.0,602,2021-04-30/23:52:51,2021-05-01/00:28:09
85000,2384,Ringtee,ERM,6.69,1973.0,473,2021-04-30/23:53:50,2021-05-01/00:15:46
85001,2227,Kalda tee,Põhja puiestee,1.22,1940.0,340,2021-04-30/23:53:53,2021-04-30/23:58:42
85002,2537,Põhja puiestee,Sõpruse sild,2.35,NaN,NaN,2021-04-30/23:54:30,2021-05-01/00:02:03


In [48]:

x['unlocktime']  = pd.to_datetime(x['unlocktime'] ,errors='coerce', format="%Y-%m-%d/%H:%M:%S")

x['locktime']  = pd.to_datetime(x['locktime'] ,errors='coerce', format="%Y-%m-%d/%H:%M:%S")
x

,cyclenumber,startstationname,endstationname,length,yearOfBirth,first3IdNumber,unlocktime,locktime
0,2581,Aparaaditehas,Rahu,1.73,NaN,601,2021-01-01 06:00:21,2021-01-01 06:10:52
1,2544,Uueturu,Vabadussild,1.10,1997.0,397,2021-01-01 06:21:41,2021-01-01 06:27:36
2,2715,Põhja puiestee,Mõisavahe,2.50,2003.0,603,2021-01-01 07:04:00,2021-01-01 07:24:56
3,2716,Põhja puiestee,Mõisavahe,2.40,2003.0,603,2021-01-01 07:04:31,2021-01-01 07:24:52
4,2548,Turusild,Sõpruse sild,1.60,NaN,499,2021-01-01 07:08:03,2021-01-01 07:17:57
...,...,...,...,...,...,...,...,...
84999,2514,Kalda tee,Kalda tee,6.82,2002.0,602,2021-04-30 23:52:51,2021-05-01 00:28:09
85000,2384,Ringtee,ERM,6.69,1973.0,473,2021-04-30 23:53:50,2021-05-01 00:15:46
85001,2227,Kalda tee,Põhja puiestee,1.22,1940.0,340,2021-04-30 23:53:53,2021-04-30 23:58:42
85002,2537,Põhja puiestee,Sõpruse sild,2.35,NaN,NaN,2021-04-30 23:54:30,2021-05-01 00:02:03


In [50]:
print(x.loc[85003]["unlocktime"]-x.loc[85003]["locktime"])

-1 days +23:50:38


,cyclenumber,unlockedat,unlockedattime,lockedat,lockedattime,startstationserialnumber,startstationname,endstationserialnumber,endstationname,yearOfBirth,first3IdNumber
length,,,,,,,,,,,
0.00,62518,62518,62518,62518,62518,62518,62518,58479,62518,33951,47992
0.01,15571,15571,15571,15571,15571,15571,15571,15523,15571,8539,11727
0.02,5394,5394,5394,5394,5394,5394,5394,5385,5394,2928,4067
0.03,3804,3804,3804,3804,3804,3804,3804,3784,3804,2027,2895
0.04,3546,3546,3546,3546,3546,3546,3546,3542,3546,1819,2753
0.05,2450,2450,2450,2450,2450,2450,2450,2444,2450,1275,1945
0.06,2158,2158,2158,2158,2158,2158,2158,2149,2158,1131,1658
0.07,2133,2133,2133,2133,2133,2133,2133,2130,2133,1121,1658
0.08,1325,1325,1325,1325,1325,1325,1325,1323,1325,691,1011


In [18]:
print(data2020.loc[213319]["unlockedattime"]-data2020.loc[213319]["lockedattime"])

TypeError: unsupported operand type(s) for -: 'str' and 'str'